In [1]:
# library imports 
import sys
from importlib import reload
#import findspark
import customHelpers as helper
#findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType,BooleanType,DateType

reload(helper)

<module 'customHelpers' from '/Users/luisedumuller/Documents/Studies/MDS/CloudComputing/Assignment2/COMP5349_AmazonProductReviewAnalysis/notebooks/customHelpers.py'>

In [2]:
# initialise the session 

spark = SparkSession \
    .builder \
    .appName("Amazon Product Review Analysis") \
    .getOrCreate()

## Load Dataset 

| Column | Description | 
| :--- | :--- |
| marketplace | 2 letter country code of the marketplace where the review was written. |
| customer_id | Random identifier that can be used to aggregate reviews written by a single author. |
| review_id | The unique ID of the review. |
| product_id | The unique Product ID the review pertains to. In the multilingual dataset the reviews for the same product in different countries can be grouped by the same product_id. | 
| product_parent | Random identifier that can be used to aggregate reviews for the same product. |
| product_title | Title of the product. | 
| product_category | Broad product category that can be used to group reviews (also used to group the dataset into  coherent parts). | 
| star_rating | the 1-5 star rating of the review. | 
| helpful_votes | Number of helpful votes. | 
| total_votes | Number of total votes the review received. | 
| vine | Review was written as part of the Vine program. |
| verified_purchase | The review is on a verified purchase. |
| review_headline | The title of the review. |
| review_body | The review text. |
| review_date | The date the review was written | 


DATA FORMAT
Tab ('\t') separated text file, without quote or escape characters.
First line in each file is header; 1 line corresponds to 1 record.

In [3]:
# load the data set 
#review_data = '../data/sample_us.tsv'
# actual data load - PERFORMANCE WARNING ON LOCAL MACHINE
review_data = '../data/amazon_reviews_us_Music_v1_00.tsv'

aws_product_review_schema = StructType([
    StructField("marketplace", StringType(), True),
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id",StringType(),True),
    StructField("product_parent",StringType(),False),
    StructField("product_title", StringType(), False),
    StructField("product_category", StringType(), False),
    StructField("star_rating", IntegerType(), False),
    StructField("helpful_votes",IntegerType(),False),
    StructField("total_votes", IntegerType(), False),
    StructField("vine",StringType(),False),
    StructField("verified_purchase", StringType(), False),
    StructField("review_headline", StringType(), False),
    StructField("review_body", StringType(), False),
    StructField("review_date",DateType(),False)])

aws_product_review_schema_limited = StructType([
    StructField("customer_id", StringType(), True),
    StructField("review_id", StringType(), True),
    StructField("product_id",StringType(),True),
    StructField("product_title", StringType(), False),
    StructField("product_category", StringType(), False),
    StructField("star_rating", IntegerType(), False),
    StructField("helpful_votes",IntegerType(),False),
    StructField("total_votes", IntegerType(), False),
    StructField("review_headline", StringType(), False),
    StructField("review_body", StringType(), False),
    StructField("review_date",DateType(),False)])

%time awsProductReview_raw_data = spark.read.csv(review_data,header=True,sep="\t",schema=aws_product_review_schema)


CPU times: user 2.04 ms, sys: 1.11 ms, total: 3.15 ms
Wall time: 1.89 s


In [4]:
# when testing in local machine only 
print(awsProductReview_raw_data.count())
# limit to 1 mil
awsProductReview_raw_data = awsProductReview_raw_data.limit(100000)
print(awsProductReview_raw_data.count())

4751577
100000


In [5]:
dfProductReview = awsProductReview_raw_data.drop('vine').drop('verified_purchase') \
                    .drop('product_parent').drop('marketplace')

In [6]:
# remove rows with no review text 
print("number of rows before filter: {0}".format(dfProductReview.count()))
dfFilteredReviews = dfProductReview.na.drop(subset=["review_body"])
print("number of rows after filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 100000
number of rows after filter: 99977


## Stage One: Overall statistics

### Produce overall summary statistics of the data set, in particular,

* the total number of reviews
* the number of unique users
* the number of unique products

In [7]:
from pyspark.sql.functions import col, count, countDistinct, when, isnull

dfOverallStats = dfFilteredReviews.agg(countDistinct("customer_id").alias("unique_customers"), \
                    countDistinct("product_id").alias("unique_products"), \
                    count(col="review_id").alias("total_reviews")) \

dfOverallStats.show()

+----------------+---------------+-------------+
|unique_customers|unique_products|total_reviews|
+----------------+---------------+-------------+
|           57062|          58745|        99977|
+----------------+---------------+-------------+



In [8]:
# CHECK IN ALL THE COLUMNS IF THERE IS ANY NULL VALUE.
dfFilteredReviews.select([count(when(isnull(column), column)).alias(column) \
                        for column in dfProductReview.columns]).show()

+-----------+---------+----------+-------------+----------------+-----------+-------------+-----------+---------------+-----------+-----------+
|customer_id|review_id|product_id|product_title|product_category|star_rating|helpful_votes|total_votes|review_headline|review_body|review_date|
+-----------+---------+----------+-------------+----------------+-----------+-------------+-----------+---------------+-----------+-----------+
|          0|        0|         0|            0|               0|          0|            0|          0|              0|          0|          0|
+-----------+---------+----------+-------------+----------------+-----------+-------------+-----------+---------------+-----------+-----------+



### For user-review distribution, you are asked to find out:

* the largest number of reviews published by a single user
* the top 10 users ranked by the number of reviews they publish
* the median number of reviews published by a user

In [9]:
from pyspark.sql.window import Window
from pyspark.sql.functions import count
import pyspark.sql.functions as F

dfUserReviewCounts = helper.distributionStats(dfRecords=dfFilteredReviews.select("customer_id", "review_id"), \
                                              partitionBy="customer_id",countBy="review_id", \
                                              returnCountName="total_reviews")
print("Top Reviewer:")
dfUserReviewCounts.show(1)
print("Top 10 Reviewers:")
dfUserReviewCounts.show(10)

user_review_median=dfUserReviewCounts.approxQuantile("total_reviews", [0.50], 0)[0]
print("median number of {0} reviews published by user".format(user_review_median))

Top Reviewer:
+-----------+-------------+
|customer_id|total_reviews|
+-----------+-------------+
|   15536614|          373|
+-----------+-------------+
only showing top 1 row

Top 10 Reviewers:
+-----------+-------------+
|customer_id|total_reviews|
+-----------+-------------+
|   15536614|          373|
|   38214553|          226|
|    4276914|          222|
|    8342883|          194|
|    5291529|          190|
|   13634768|          180|
|    2112938|          126|
|   38229524|           98|
|   48046800|           97|
|   22716027|           92|
+-----------+-------------+
only showing top 10 rows

median number of 1.0 reviews published by user


### For product-review distribution, you are asked to find out:
    
* the largest number of reviews written for a single product
* the top 10 products ranked by the number of reviews they have
* the median number of reviews a product has

In [10]:
dfProductReviewCounts = helper.distributionStats(dfRecords=dfProductReview.select("product_id", "review_id"), \
                                              partitionBy="product_id",countBy="review_id", \
                                              returnCountName="total_reviews")
print("Top Product By Review:")
dfProductReviewCounts.show(1)
print("Top 10 Products by Reviews:")
dfProductReviewCounts.show(10)

product_review_median=int(dfProductReviewCounts.approxQuantile("total_reviews", [0.50], 0)[0])
print("median number of {0} reviews per product".format(product_review_median))

Top Product By Review:
+----------+-------------+
|product_id|total_reviews|
+----------+-------------+
|B00VXGTJMU|          256|
+----------+-------------+
only showing top 1 row

Top 10 Products by Reviews:
+----------+-------------+
|product_id|total_reviews|
+----------+-------------+
|B00VXGTJMU|          256|
|B00VMRJPCE|          255|
|B00UCFVIDQ|          205|
|B00VMQK37Q|          170|
|B00ZQUP38S|          158|
|B00WE2SMKC|          139|
|B00WSOWR0M|          120|
|B00MRHANNI|          116|
|B00VTBBEL8|          115|
|B00XJJAWES|          112|
+----------+-------------+
only showing top 10 rows

median number of 1 reviews per product


## Stage Two: Filtering Unwanted Data

filter reviews based on length, reviewer and product feature. In particular, the following reviews should be removed:

* reviews with less than two sentences in the review body.
* reviews published by users with less than median number of reviews published
* reviews from products with less than median number of reviews received

NOTE: Sentence Segmentation Using: NLTK

In [11]:
# reviews with less than 2 sentences in review_body
# convert to RDD and carry out a filter to remove rows with less than 2 sentences 

print("number of rows before filter: {0}".format(dfFilteredReviews.count()))

dfFilteredReviews = dfFilteredReviews.filter(helper.FilterSentences('review_body'))

dfFilteredReviews.show(1)
dfFilteredReviews.cache()

print("number of rows post filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 99977
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|customer_id|     review_id|product_id|       product_title|product_category|star_rating|helpful_votes|total_votes|     review_headline|         review_body|review_date|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|Same Trailer Diff...|           Music|          5|            0|          0|A new fave in our...|This is the album...| 2015-08-31|
+-----------+--------------+----------+--------------------+----------------+-----------+-------------+-----------+--------------------+--------------------+-----------+
only showing top 1 row

number of rows post filter: 28955


In [12]:
# user review filter 
print("number of rows before filter: {0}".format(dfFilteredReviews.count()))

window = Window.partitionBy("customer_id")
dfFilteredReviews = dfFilteredReviews \
    .withColumn("review_count", count("review_id") \
    .over(window)) \
    .filter(col("review_count") >= user_review_median) \
    .drop("review_count")

print("number of rows post filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 28955
number of rows post filter: 28955


In [13]:
# product review filter
print("number of rows before filter: {0}".format(dfFilteredReviews.count()))

window = Window.partitionBy("product_id")
dfFilteredReviews = dfFilteredReviews \
    .withColumn("review_count", count("review_id") \
    .over(window)) \
    .filter(col("review_count") >= product_review_median) \
    .drop("review_count")

print("number of rows post filter: {0}".format(dfFilteredReviews.count()))

number of rows before filter: 28955
number of rows post filter: 28955


In [14]:
print("Original Number of Rows before cleanup: {0}".format(dfProductReview.count()))
print("Number of rows after all filters applied: {0}".format(dfFilteredReviews.count()))

Original Number of Rows before cleanup: 100000
Number of rows after all filters applied: 28955


In [15]:
dfFilteredReviews.cache()

DataFrame[customer_id: string, review_id: string, product_id: string, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, review_headline: string, review_body: string, review_date: date]

#### After filtering out the above, find out:

* top 10 users ranked by median number of sentences in the reviews they have published
* top 10 products ranked by median number of sentences in the reviews they have received

In [16]:
# top 10 users ranked by median number of sentences in the reviews they have published
dfTop10UsersBySents = helper.getTopBySentMedian(dfRecords=dfFilteredReviews, \
                                                partitionBy="customer_id", \
                                                textCol="review_body",\
                                                medianColName="median_sents",n=10)
dfTop10UsersBySents.show()

+-----------+------------+
|customer_id|median_sents|
+-----------+------------+
|   49745257|         241|
|   49754397|         239|
|   14678937|         155|
|   22434772|         127|
|   20894201|         103|
|   25007515|          87|
|    1459729|          82|
|   51979520|          82|
|   49758023|          79|
|   34564717|          77|
+-----------+------------+



In [17]:
# top 10 products ranked by median number of sentences in the reviews they have received
dfTop10ProductsBySents = helper.getTopBySentMedian(dfRecords=dfFilteredReviews, \
                                                   partitionBy="product_id", \
                                                   textCol="review_body", \
                                                   medianColName="median_sents",n=10)

dfTop10ProductsBySents.show(10)

+----------+------------+
|product_id|median_sents|
+----------+------------+
|B003ZUCXP2|         217|
|B010SOIC5C|         194|
|B00Q9H9GBM|         151|
|B003UUQ7OK|         150|
|B000007WPE|         148|
|B00FY3X5GO|         146|
|B00FG1EVUS|         145|
|B003XD03DU|         134|
|B012IV1E62|         134|
|B00GG3JEU2|         134|
+----------+------------+



In [18]:
# save the cleaned and filtered dataframe to file system 

#dfFilteredReviews.coalesce(1).write.format("parquet") \
#    .option("header", "true").saveAsTable('filteredReviews',mode="overwrite")
dfFilteredReviews.coalesce(1).write.csv("../output",mode="overwrite",header=True,sep="\t")

## Stage 3 Similarity analysis with Sentence Embedding

perform similarity analysis on the review sentences. The analysis involves segmenting review body into multiple sentences; encoding each sentence as vector so that the distance between pair of sentences can be computed.

### Positive vs. Negative Reviews

* pick a product from the top 10 products in stage 1
* Create a positive and negative class of reviews using the rating 
    - Positive Class - rate >=4 
    - Negative Class - rate <= 2
    - for each review, extracting the review body part and segment it into multiple sentences.
    - encode the sentences using google universal encoder

In [19]:
filtered_data = "../output/part-*.csv"
dfBaseDataset = spark.read.csv(filtered_data,header=True,sep="\t",schema=aws_product_review_schema_limited)

In [20]:
# product from top 10 by review number 
base_product_id = "B00MIA0KGY"

dfSelectedProduct = dfBaseDataset.where((col("product_id") == base_product_id))
dfPositiveClass = dfBaseDataset.where((col("product_id") == base_product_id) & (col("star_rating") >= 4))
dfNegativeClass = dfBaseDataset.where((col("product_id") == base_product_id) & (col("star_rating") <= 2))

In [21]:
print("number of reviews from {0}: {1}".format(base_product_id, dfSelectedProduct.count()))
print("number of positives reviews from {0}: {1}".format(base_product_id,dfPositiveClass.count()))
print("number of negatives reviews from {0}: {1}".format(base_product_id,dfNegativeClass.count()))



number of reviews from B00MIA0KGY: 14
number of positives reviews from B00MIA0KGY: 11
number of negatives reviews from B00MIA0KGY: 1


### Extract the sentences - similar to flatMap

In [22]:
# for each review, extract the review body part and segment it into multiple sentences
# extract the positive sentences
dfPosSents = dfPositiveClass.select("review_id","review_body") \
    .withColumn("sentences", helper.GenerateSentences("review_body")) \
    .select("review_id", F.explode_outer("sentences").alias("sentence"))

print("number of sentences from positive reviews: {0}".format(dfPosSents.count()))

number of sentences from positive reviews: 75


In [23]:
# extract the negative sentences
dfNegSents = dfNegativeClass.select("review_id","review_body") \
    .withColumn("sentences", helper.GenerateSentences("review_body")) \
    .select("review_id", F.explode_outer("sentences").alias("sentence"))

print("number of sentences from negative reviews: {0}".format(dfNegSents.count()))

dfNegSents.show()

number of sentences from negative reviews: 4
+--------------+--------------------+
|     review_id|            sentence|
+--------------+--------------------+
|R3VTZ9CGH2A2BZ|I thought I would...|
|R3VTZ9CGH2A2BZ|Didn't enjoy a si...|
|R3VTZ9CGH2A2BZ|Her Duets album w...|
|R3VTZ9CGH2A2BZ|I'll be REselling...|
+--------------+--------------------+



### Encoding the sentences - google universal encoder 

In [25]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from pyspark.sql.types import StructType, StructField, StringType,IntegerType, FloatType,BooleanType,DateType,ArrayType


In [95]:
# Processing the data for embedding: first step selecting the sentence column from the 
# data frames sentences. not sure why it doesn't work if don't use '.limit()' even if
# dataframe is short 
pos_rev_text = dfPosSents.limit(1000).select('sentence')
#second: convert the dataframe to rdd pipeline
pos_rev_clean_text_rdd = pos_rev_text.rdd.map(lambda row: str(row[0])) \
                    .filter(lambda data: data is not None).cache()

neg_rev_text = dfNegSents.limit(1000).select('sentence')
#second: convert the dataframe to rdd pipeline
neg_rev_clean_text_rdd = neg_rev_text.rdd.map(lambda row: str(row[0])) \
                    .filter(lambda data: data is not None).cache()

In [96]:
## function from lab 9
def review_embed(rev_text_partition):
    module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]
    embed = hub.Module(module_url)
    # mapPartition would supply element inside a partition using generator stype
    # this does not fit tensorflow stype
    rev_text_list = [text for text in rev_text_partition]
    with tf.Session() as session:
        session.run([tf.global_variables_initializer(), tf.tables_initializer()])
        message_embeddings = session.run(embed(rev_text_list))
    
    return message_embeddings



In [97]:
# getting the embedding from sentences
positive_review_embedding = pos_rev_clean_text_rdd.mapPartitions(review_embed).cache()
negative_review_embedding = neg_rev_clean_text_rdd.mapPartitions(review_embed).cache()


In [98]:
negative_review_embedding.first()

array([-2.29360759e-02, -7.71128610e-02,  1.08671496e-02, -2.26527657e-02,
       -1.42055964e-02,  4.51085623e-03, -2.69437823e-02, -6.63384870e-02,
        2.49271002e-02,  3.28172594e-02,  7.84407631e-02, -2.33665463e-02,
       -5.13132662e-02,  8.24994445e-02,  2.22025961e-02,  2.84204148e-02,
       -8.21404681e-02,  3.57403373e-03,  1.73447020e-02, -2.06466485e-02,
        6.40797466e-02, -9.52492207e-02,  1.36482595e-02, -4.48517874e-02,
        1.92933846e-02,  4.03569601e-02, -4.24939431e-02, -8.80729407e-02,
       -5.51371314e-02,  2.85780821e-02, -7.24689336e-03,  4.83036302e-02,
        1.93823371e-02,  6.25806227e-02,  5.95747046e-02,  1.31674418e-02,
       -4.57846038e-02,  4.29204926e-02,  1.27157327e-02, -2.83983871e-02,
        4.56347391e-02,  7.72189535e-03, -6.26320466e-02, -2.02600919e-02,
       -8.60578381e-03, -6.87528262e-03,  8.81018955e-03, -4.72780019e-02,
        1.41477585e-02,  6.87720478e-02, -1.73657518e-02,  1.34534035e-02,
       -5.27658761e-02, -

### Intra-Class Similarity

We want to find out if sentences in the same category are closely related with each other. The closeness is measured by average distance between points in the class. In our case, point refers to the sentence encoding and pair-wise distance is measured by Cosine distance. Cosine distance is computed as “1 − CosineSimilarity”. It has a value between 0 and 2.


In [99]:
# calculate the distances between vectors and average the results.
from pyspark.ml.linalg import Vectors

def CosineDistance(embedded_sentences):
    
    # transform the numpy arrays into pyspark Vectors
    embedded_sentences_vect = embedded_sentences.map(Vectors.dense).cache()
    
    # collect the cartesian combination of the vectors
    # eg: (vect1, vect1), (vect1, vect2),(vect1, vect3), etc... 
    # I am using cartesian here to avoid nested for loops in the Cosine Distance calculation.
    cartesian_vector = embedded_sentences_vect.cartesian(embedded_sentences_vect).collect()
    
    distances = []
    for v in cartesian_vector:
        if v[0] != v[1]:
            distance = 1 - (v[0].dot(v[1])/(v[0].norm(2)*v[1].norm(2)))
            distances.append(distance)

    return sum(distances)/len(distances)

In [69]:
pos_dist = CosineDistance(positive_review_embedding)
print("Average cosine distance between positive reviews: {0}".format(pos_dist))

Average cosine distance between positive reviews: 0.6659711894002901


In [101]:
neg_dist = CosineDistance(negative_review_embedding)
print("Average cosine distance between negative reviews: {0}".format(neg_dist))

Average cosine distance between negative reviews: 0.6777541090667892


### Class Center Sentences

Find out the class center and its 10 closest neighbours for positive and negative class respectively. We define class center as the point that has the smallest average distance to other points in the class. Again in this case point refers to the sentence encoding and pair-wise distance are measured by Cosine distance.
The result should show the text of the center sentence, the review id it belongs to and its 10 closest neighbouring sentences text and their respective review id.